# Project

# Notebook Overview: kNN k-Nearest Neighbors


This particular notebook demonstrates an application of k-Nearest Neighbors on a truncated dataset. 

The data looks at successfully landed stats for the first instance of a B_fighter. 

-Ultimately, this means that we have to eventually modify the data in that we average of that fighter's record. For example, if Khabib Nurmagomedov, over three matches, had 5,8,3 takedowns, then possibly we can modify that record into 
the average over those instances (meaning: Khabib Nurmagomedov Takedown: 5.3 as opposed to Khabib 3, Khabib 5, Khabib 3). Or double check the cvs zip to see if maybe another dataset offered is more beneficial. 

-Does not cluster by weight class. Currently, weight class is represented by a string, which I have to go through and change to be the numeric value

-Does not cluster by gender. Need to add a bool value row which would indicate if the fighter is male or female. 
This can be filtered by using the weight_class's string, which says "Women's Bantamweight" vs "Bantamweight". 

-TODO: 
In the fighter_indeces, need to get the index that corresponds with the fighter. 
ex: (def fighter_index (fighter) =..... , fighter_index("Amanda Nunes") = 80) 

Need to produces the recommendations list for that fighter using fighter_index 
ex: (def getRecommendationsfor("Amanda Nunes"):.....", getRecommendationsfor("Amanda Nunes") = "Valentina Schevchenko"). 

Note: Schevchenko's just a guess, but more than likely, they would be similar fighters. 



Application starts at In[48]. 

## Data Cleaning and Imputation

We begin by loading the dataset, and showing a few of the matches to prove the data has been loaded.

In [1]:
import pandas as pd
df = pd.read_csv("data.csv")
df.head(5)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019-06-08,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0.0,...,2.0,0.0,0.0,8.0,Orthodox,162.56,162.56,135.0,31.0,32.0
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019-06-08,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0.0,...,0.0,2.0,0.0,5.0,Southpaw,165.10,167.64,125.0,32.0,31.0
2,Tony Ferguson,Donald Cerrone,Dan Miragliotta,2019-06-08,"Chicago, Illinois, USA",Red,False,Lightweight,3,0.0,...,3.0,6.0,1.0,14.0,Orthodox,180.34,193.04,155.0,36.0,35.0
3,Jimmie Rivera,Petr Yan,Kevin MacDonald,2019-06-08,"Chicago, Illinois, USA",Blue,False,Bantamweight,3,0.0,...,1.0,0.0,0.0,6.0,Orthodox,162.56,172.72,135.0,26.0,29.0
4,Tai Tuivasa,Blagoy Ivanov,Dan Miragliotta,2019-06-08,"Chicago, Illinois, USA",Blue,False,Heavyweight,3,0.0,...,2.0,0.0,0.0,3.0,Southpaw,187.96,190.50,264.0,32.0,26.0


### Missing Stances and Referee

Some of the matches feature fighters with missing stance information or missing referee information. Since this is a relatively small number of matches out of the whole dataset, we will drop these matches from our consideration.

In [2]:
# Show number of matches in dataset before removing matches with missing stance information
print('Number of matches prior to filtering: ' + str(len(df)))

# Remove matches with missing stance information
filter1 = df[df['B_Stance'].notnull()]
filter2 = filter1[filter1['R_Stance'].notnull()]
filter3 = filter2[filter2['Referee'].notnull()]
df = filter3
print('Number of matchs after filtering: ' + str(len(filter3)))

Number of matches prior to filtering: 5144
Number of matchs after filtering: 4865


### Missing Numerical Data

Many of the rows contain missing numerical data in certain columns. We will fill in these columns with the median for that column.

First we will find columns with missing numerical data to demonstrate that our imputation is successful.

In [3]:
df.columns[df.isnull().any()]

Index(['B_avg_BODY_att', 'B_avg_BODY_landed', 'B_avg_CLINCH_att',
       'B_avg_CLINCH_landed', 'B_avg_DISTANCE_att', 'B_avg_DISTANCE_landed',
       'B_avg_GROUND_att', 'B_avg_GROUND_landed', 'B_avg_HEAD_att',
       'B_avg_HEAD_landed',
       ...
       'R_avg_opp_SUB_ATT', 'R_avg_opp_TD_att', 'R_avg_opp_TD_landed',
       'R_avg_opp_TD_pct', 'R_avg_opp_TOTAL_STR_att',
       'R_avg_opp_TOTAL_STR_landed', 'R_total_time_fought(seconds)',
       'R_Reach_cms', 'B_age', 'R_age'],
      dtype='object', length=104)

We select R_age for our demonstration. Now we find a few rows that have a missing R_age.

In [4]:
people = df[df['R_age'].isnull()].head(5)
people

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4171,Per Eklund,Samy Schiavo,Leon Roberts,2008-10-18,"Birmingham, England, United Kingdom",Red,False,Lightweight,3,1.0,...,0.0,0.0,0.0,0.0,Orthodox,177.80,182.88,155.0,32.0,NaN
4376,Jess Liaudin,Anthony Torres,Mario Yamasaki,2007-09-08,"London, England, United Kingdom",Red,False,Welterweight,3,0.0,...,0.0,1.0,0.0,1.0,Orthodox,175.26,182.88,170.0,29.0,NaN
4438,Jess Liaudin,Dennis Siver,Steve Mazzagatti,2007-04-21,"Manchester, England, United Kingdom",Red,False,Welterweight,3,0.0,...,0.0,0.0,0.0,0.0,Orthodox,175.26,182.88,170.0,28.0,NaN
4767,Keith Rockel,Chris Liguori,John McCarthy,2003-11-21,"Uncasville, Connecticut, USA",Red,False,Middleweight,3,0.0,...,0.0,0.0,0.0,0.0,Orthodox,182.88,NaN,185.0,NaN,NaN
4908,Ben Earwood,Chris Lytle,Mario Yamasaki,2000-11-17,"Atlantic City, New Jersey, USA",Red,False,Welterweight,2,0.0,...,0.0,0.0,0.0,0.0,Orthodox,172.72,NaN,170.0,26.0,NaN


Now we find the median for that column.

In [5]:
df['R_age'].median()

29.0

Now we apply fillna to the entire dataset, and show that the column is filled in with the median.

In [6]:
df = df.fillna(df.median())
df.loc[people.index.tolist()]

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4171,Per Eklund,Samy Schiavo,Leon Roberts,2008-10-18,"Birmingham, England, United Kingdom",Red,False,Lightweight,3,1.0,...,0.0,0.0,0.0,0.0,Orthodox,177.80,182.88,155.0,32.0,29.0
4376,Jess Liaudin,Anthony Torres,Mario Yamasaki,2007-09-08,"London, England, United Kingdom",Red,False,Welterweight,3,0.0,...,0.0,1.0,0.0,1.0,Orthodox,175.26,182.88,170.0,29.0,29.0
4438,Jess Liaudin,Dennis Siver,Steve Mazzagatti,2007-04-21,"Manchester, England, United Kingdom",Red,False,Welterweight,3,0.0,...,0.0,0.0,0.0,0.0,Orthodox,175.26,182.88,170.0,28.0,29.0
4767,Keith Rockel,Chris Liguori,John McCarthy,2003-11-21,"Uncasville, Connecticut, USA",Red,False,Middleweight,3,0.0,...,0.0,0.0,0.0,0.0,Orthodox,182.88,185.42,185.0,29.0,29.0
4908,Ben Earwood,Chris Lytle,Mario Yamasaki,2000-11-17,"Atlantic City, New Jersey, USA",Red,False,Welterweight,2,0.0,...,0.0,0.0,0.0,0.0,Orthodox,172.72,185.42,170.0,26.0,29.0


### Dimensionality Reduction-PCA ### 

We apply dimensionality reduction to obtain an ordered list of components that account for the largest variance in the data set in order to ultimately group similar fighters based on their fighting styles



Firstly, we drop the non numeric columns

# kNN 

In [134]:
#dealing with only the B_fighter and successful attacks landed. Much smaller dataset. 
#reasoning: successful attacks tend are more significatn 
#attacks that were attempted showcase "volume" of that fight which vary match by match
#whereas successfully landed shots showcase a fighter's accuracy/overall control of the match

view = df[["B_fighter", "weight_class", "R_current_lose_streak", "B_current_win_streak", "B_draw", "B_avg_BODY_landed",
           "B_avg_CLINCH_landed", "B_avg_DISTANCE_landed", "B_avg_GROUND_landed", "B_avg_HEAD_landed", "B_avg_KD", 
           "B_avg_LEG_landed", "B_avg_PASS", "B_avg_REV", "B_avg_SIG_STR_landed", "B_avg_SIG_STR_pct", "B_avg_SUB_ATT", 
           "B_avg_TD_landed", "B_avg_TD_pct", "B_avg_TOTAL_STR_landed", "B_longest_win_streak", "B_losses", 
           "B_avg_opp_BODY_landed", "B_avg_opp_CLINCH_landed", "B_avg_opp_DISTANCE_landed", "B_avg_opp_GROUND_landed", 
           "B_avg_opp_HEAD_landed", "B_avg_opp_KD", "B_avg_opp_LEG_landed", "B_avg_opp_PASS", "B_avg_opp_REV", 
           "B_avg_opp_SIG_STR_landed", "B_avg_opp_SIG_STR_pct", "B_avg_opp_TD_landed", "B_avg_opp_TD_pct",
           "B_avg_opp_TOTAL_STR_landed", "B_total_title_bouts", "B_win_by_Decision_Majority", "B_win_by_Decision_Split", 
           "B_win_by_Decision_Unanimous", "B_win_by_KO/TKO", "B_win_by_Submission", "B_win_by_TKO_Doctor_Stoppage", 
           "B_wins"]]



#dropping duplicates
#newView = view.drop_duplicates(subset = "B_fighter", keep = "first")

newView.head()

,B_fighter,weight_class,R_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_landed,B_avg_CLINCH_landed,B_avg_DISTANCE_landed,B_avg_GROUND_landed,B_avg_HEAD_landed,...,B_avg_opp_TD_pct,B_avg_opp_TOTAL_STR_landed,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins
0,Marlon Moraes,Bantamweight,0.0,4.0,0.0,6.000000,0.000000,20.600000,2.000000,11.200000,...,0.100000,19.200000,0.0,0.0,1.0,0.0,2.0,1.0,0.0,4.0
1,Jessica Eye,Women's Flyweight,0.0,3.0,0.0,9.100000,7.300000,42.100000,1.900000,32.000000,...,0.231000,75.400000,0.0,0.0,2.0,1.0,0.0,0.0,1.0,4.0
2,Donald Cerrone,Lightweight,0.0,3.0,0.0,11.322581,4.387097,38.580645,3.806452,23.258065,...,0.063548,49.774194,1.0,0.0,0.0,7.0,10.0,6.0,0.0,23.0
3,Petr Yan,Bantamweight,1.0,4.0,0.0,14.000000,11.000000,48.750000,10.500000,53.750000,...,0.097500,34.250000,0.0,0.0,0.0,2.0,2.0,0.0,0.0,4.0
4,Blagoy Ivanov,Heavyweight,1.0,1.0,0.0,14.500000,2.000000,59.500000,0.000000,45.000000,...,0.000000,90.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [119]:
#changing the weight class from string to numeric: 
#incomplete because identified as "women's" and "mens"
#newView["weight_class"].unique()

# for i in newView["weight_class"]:
#     if i == "Bantamweight": 
#         df.loc[df['weight_class'] == i] = 135
    
# display(newView["weight_class"])


#as of right now, leaving as is. 


In [135]:
newView[['B_fighter', "B_wins"]].loc[newView['B_fighter'] == "Khabib Nurmagomedov"]


,B_fighter,B_wins
2478,Khabib Nurmagomedov,5.0


In [136]:
from sklearn.neighbors import NearestNeighbors
#later after reformatting weight_class to have the numerical numbers
#we will add it in. for now, drop since string. 
stats = newView.drop(columns = ["B_fighter", "R_current_lose_streak", "weight_class"])

In [137]:
stats.head()

,B_current_win_streak,B_draw,B_avg_BODY_landed,B_avg_CLINCH_landed,B_avg_DISTANCE_landed,B_avg_GROUND_landed,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_landed,B_avg_PASS,...,B_avg_opp_TD_pct,B_avg_opp_TOTAL_STR_landed,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins
0,4.0,0.0,6.000000,0.000000,20.600000,2.000000,11.200000,0.800000,5.400000,0.400000,...,0.100000,19.200000,0.0,0.0,1.0,0.0,2.0,1.0,0.0,4.0
1,3.0,0.0,9.100000,7.300000,42.100000,1.900000,32.000000,0.000000,10.200000,0.800000,...,0.231000,75.400000,0.0,0.0,2.0,1.0,0.0,0.0,1.0,4.0
2,3.0,0.0,11.322581,4.387097,38.580645,3.806452,23.258065,0.645161,12.193548,0.935484,...,0.063548,49.774194,1.0,0.0,0.0,7.0,10.0,6.0,0.0,23.0
3,4.0,0.0,14.000000,11.000000,48.750000,10.500000,53.750000,0.500000,2.500000,0.500000,...,0.097500,34.250000,0.0,0.0,0.0,2.0,2.0,0.0,0.0,4.0
4,1.0,0.0,14.500000,2.000000,59.500000,0.000000,45.000000,0.000000,2.000000,0.000000,...,0.000000,90.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [159]:
knn = NearestNeighbors(metric = "cosine", algorithm = "brute", n_neighbors=5)
knn.fit(stats)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [160]:
dist, fighter_indices = knn.kneighbors(stats)
fighter_indices[40]

array([  40,   31,  242,  104, 1027])

In [163]:
#finding similar recommendations for Amanda Nunes and Khabib Nurmagomedov

def get_recommends(fighter):
    stats_rindex = stats.reset_index(drop=True)
    newView_rindex = newView.reset_index(drop=True)
    dist, fighter_indices = knn.kneighbors(stats_rindex)
    fighters = fighter_indices[newView_rindex.loc[newView_rindex['B_fighter'] == fighter].index].tolist()[0]
    fighters_as_df = newView_rindex[newView_rindex.index.isin(fighters)]
    fighters_as_df = fighters_as_df[fighters_as_df['B_fighter'] != fighter]
    return fighters_as_df

get_recommends("Amanda Nunes")

,B_fighter,weight_class,R_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_landed,B_avg_CLINCH_landed,B_avg_DISTANCE_landed,B_avg_GROUND_landed,B_avg_HEAD_landed,...,B_avg_opp_TD_pct,B_avg_opp_TOTAL_STR_landed,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins
146,Diego Ferreira,Lightweight,0.0,3.0,0.0,6.428571,2.857143,20.428571,8.285714,22.285714,...,0.171429,27.857143,0.0,0.0,0.0,1.0,3.0,1.0,0.0,5.0
647,Frankie Edgar,Featherweight,1.0,5.0,0.0,12.210526,5.789474,40.368421,16.315789,41.263158,...,0.204737,50.684211,7.0,0.0,0.0,8.0,4.0,2.0,0.0,14.0
780,Fabricio Werdum,Heavyweight,0.0,5.0,0.0,6.666667,5.333333,31.111111,6.444444,28.111111,...,0.194444,31.000000,1.0,0.0,0.0,2.0,4.0,1.0,0.0,7.0
1132,Forrest Griffin,Light Heavyweight,1.0,2.0,0.0,7.769231,8.384615,40.076923,6.000000,31.846154,...,0.242308,43.384615,3.0,0.0,1.0,5.0,1.0,2.0,0.0,9.0


In [164]:
get_recommends("Khabib Nurmagomedov")

,B_fighter,weight_class,R_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_landed,B_avg_CLINCH_landed,B_avg_DISTANCE_landed,B_avg_GROUND_landed,B_avg_HEAD_landed,...,B_avg_opp_TD_pct,B_avg_opp_TOTAL_STR_landed,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins
120,Nad Narimani,Featherweight,0.0,2.0,0.0,7.000,7.500,35.50,21.500,50.000,...,0.00000,54.50,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0
584,Rustam Khabilov,Lightweight,0.0,3.0,0.0,2.875,2.375,13.50,8.000,17.875,...,0.23875,27.00,0.0,0.0,0.0,4.0,1.0,0.0,1.0,6.0
846,Dennis Bermudez,Featherweight,0.0,7.0,0.0,6.125,11.750,23.75,17.125,38.125,...,0.13500,48.75,1.0,0.0,2.0,2.0,1.0,2.0,0.0,7.0
977,Chris Weidman,Middleweight,0.0,5.0,0.0,2.800,3.800,16.20,8.200,21.800,...,0.00000,22.80,0.0,0.0,0.0,2.0,1.0,2.0,0.0,5.0


In [ ]:
#I need to find the index in within the figher_indices which coorespond with Amanda Nunes
#And then fight the recommended fighters. 